# Semestrální projekt z NLA1 2024/25
**Zadání č. 3**<br>
**Kučera Ondřej**

## Úkol 1
Naprogramujte v programovacím jazyce Python LU rozklad matice $A\in \mathbb{R}^{n×n}, A = LU$,
pomocí následujícího pseudokódu:

$$
\begin{array}{l}
L = I; U = 0\\
\text{for } j = 1 : n\\
    \quad\begin{array}{l}
    \text{if } j = 1\\
        \quad\begin{array}{l}
        v(j : n) = A(j : n, j)
        \end{array}\\
    \text{else}\\
        \quad\begin{array}{l}
        \text{Solve } L(1 : (j - 1), 1 : (j - 1))z = A(1 : (j - 1), j) \text{ for } z\\
            \text{ and set } U(1 : (j - 1), j) = z\\
        v(j : n) = A(j : n, j) - L(j : n, 1 : (j - 1))z
        \end{array}\\
    \text{end if}\\
    \text{if } j < n\\
        \quad\begin{array}{l}
        L((j + 1) : n, j) = v((j + 1) : n) / v(j)
        \end{array}\\
    \text{end if}\\
    U(j, j) = v(j)
    \end{array}\\
\text{end for}
\end{array}
$$

In [14]:
import numpy as np

In [15]:
def LU_rozklad(A):
    """
    Vypočítá LU rozklad čtvercové matice A.
    """
    m, n = A.shape
    if m != n:
        raise ValueError("Matice musí být čtvercová!")

    L = np.eye(n)
    U = np.zeros((n, n))
    v = np.zeros(n)

    for j in range(n):
        if j == 0:
            v[j:n] = A[j:n, j]
        else:
            z = np.zeros(j)
            for i in range(j):
                z[i] = (A[i, j] - L[i, 0:i] @ z[0:i]) / L[i, i]
            
            U[0:j, j] = z
            v[j:n] = A[j:n, j] - L[j:n, 0:j] @ z

        if j < n-1:
            L[j+1:n, j] = v[j+1:n] / v[j]
        
        U[j, j] = v[j]

    return L, U

In [16]:
A = np.array([[1, 1, 3],
              [2, 3, 5],
              [4, 4, 9]])

L, U = LU_rozklad(A)
print(L)
print(U)
print(np.allclose(A, L @ U))

[[1. 0. 0.]
 [2. 1. 0.]
 [4. 0. 1.]]
[[ 1.  1.  3.]
 [ 0.  1. -1.]
 [ 0.  0. -3.]]
True


Rozšiřte algoritmus tak, aby fungoval pro matice $A\in \mathbb{R}^{m×n}$, přičemž $m > n$.

In [17]:
def LU_rozklad(A):
    """
    Vypočítá LU rozklad obdélníkové nebo čtvercové matice A (m >= n).
    """
    m, n = A.shape
    if m < n:
        raise ValueError("Počet řádků matice nesmí být menší než počet sloupců!")

    L = np.eye(m)
    U = np.zeros((m, n))
    v = np.zeros(m)

    for j in range(n):
        if j == 0:
            v[j:m] = A[j:m, j]
        else:
            z = np.zeros(j)
            for i in range(j):
                z[i] = (A[i, j] - L[i, 0:i] @ z[0:i]) / L[i, i]
            
            U[0:j, j] = z
            v[j:m] = A[j:m, j] - L[j:m, 0:j] @ z

        if j < n-1:
            L[j+1:m, j] = v[j+1:m] / v[j]
        
        U[j, j] = v[j]
    
    for j in range(n, m):
        L[j, n-1] = (A[j, n-1] - L[j, 0:n-1] @ U[0:n-1, n-1]) / U[n-1, n-1]

    return L, U

In [18]:
A = np.array([[1, 2, -1, 0],
              [1, 4, 2, 4],
              [2, 6, 2, 2],
              [1, 4, 5, 3],
              [3, 8, 3, 3]])

L, U = LU_rozklad(A)
print(L)
print(U)
print(np.allclose(A, L @ U))

[[1. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0.]
 [2. 1. 1. 0. 0.]
 [1. 1. 3. 1. 0.]
 [3. 1. 3. 1. 1.]]
[[ 1.  2. -1.  0.]
 [ 0.  2.  3.  4.]
 [ 0.  0.  1. -2.]
 [ 0.  0.  0.  5.]
 [ 0.  0.  0.  0.]]
True


In [19]:
A = np.random.rand(10, 10)
L, U = LU_rozklad(A)
print(np.allclose(A, L@U))

A = np.random.rand(15, 10)
L, U = LU_rozklad(A)
print(np.allclose(A, L@U))

True
True


Zkuste odhadnout potřebný počet operací ve tvaru $O(kn^l)$, resp. $O(\tilde{k}{m}^{\tilde{l}})$, tzn. parametry $k, l\in \mathbb{Q}$, resp. $\tilde{k}, \tilde{l}\in \mathbb{Q}$.

### Analýza složitosti
Algoritmus se skládá z definice proměnných a dvou cyklů, z nichž ten první se provede $n$-krát a druhý se provede $(m-n)$-krát.<br>
Budu uvažovat pouze maticové/vektorové operace (násobení, sčítání/odčítání, definice/nastavení) uvnitř cyklů.

#### První cyklus
V případě $j=0$ se provede řádek 15, který nastavuje všechny prvky vektoru $v$ na prvky $j$-tého sloupce $A$. Tedy se provede $m$ operací.

V případě $j\neq 0$ se provedou řádky 17-22.
- Řádek 17 vytvoří vektor $z$ o $j$ prvcích, což je $j$ operací.
- Řádky 18-19 obsahují cyklus, který se provede $j$-krát. Řádek 19 obsahuje skalární součin dvou vektorů o $i$ prvcích, což je $i$ operací.
- Řádek 21 nastavuje část $U$ na $z$, což je $j$ operací.
- Řádek 22:
    - Nejprve se provede násobení matice s $(m-j)$ řádky a $j$ sloupci se sloupcovým vektorem o $j$ prvcích. Tohle je $(m-j)*j$ operací.
    - Výsledkem je vektor o $(m-j)$ prvcích, ten se odečte od části matice $A$ a nastaví do vektoru $v$, což je $2*(m-j)$ operací.

Dále, pro případ $j<n-1$ se provede řádek 25. V něm se nastaví část $L$ o $(m-j-1)$ prvcích, je to tedy $(m-j-1)$ operací.

V prvním cyklu je to dohromady
$$
\begin{split}
počet_{první} &= m + \sum_{j=1}^{n-1} (j + \sum_{i=0}^{j-1} i + j + (m-j)j + 2(m-j)) \\
              &= m + \sum_{j=1}^{n-1} (2j + \frac{j(j-1)}{2} + mj - j^2 + 2m - 2j) \\
              &= m + \sum_{j=1}^{n-1} (\frac{j^2-j}{2} + mj - j^2 + 2m) \\
              &= m + \sum_{j=1}^{n-1} (2m + mj - \frac{j^2}{2} - 2j) \\
              &= m + 2m(n-1) + m\frac{n(n-1)}{2} - \frac{(n-1)n(2n-1)}{6*2} - 2\frac{n(n-1)}{2} \\
              &= m + 2mn - 2m + \frac{1}{2}mn^2 - \frac{1}{2}mn - \frac{1}{6}n^3 + \frac{1}{4}n^2 - \frac{1}{12}n - n^2 + n \\
              &= \frac{1}{2}mn^2 + \frac{3}{2}mn - m - \frac{1}{6}n^3 - \frac{3}{4}n^2 + \frac{11}{12}n
\end{split}
$$
operací.

#### Druhý cyklus
Ve druhém cyklu se provádí jenom řádek 30, ve kterém se vynásobí vektory o $(n-1)$ prvcích, tedy $(n-1)$ operací.

Ve druhém cyklu se tedy dohromady provede
$$
\begin{split}
počet_{druhý} &= \sum_{j=n}^{m-1} (n-1) \\
              &= (m-n)(n-1) \\
              &= mn - m - n^2 + n
\end{split}
$$
operací.

#### Celkový počet operací
Počet operací celkem za oba cykly dostanu součtem.
$$
\begin{split}
počet_{celkem} &= počet_{první} + počet_{druhý} \\
               &= \frac{1}{2}mn^2 + \frac{3}{2}mn - m - \frac{1}{6}n^3 - \frac{3}{4}n^2 + \frac{11}{12}n + mn - m - n^2 + n \\
               &= \frac{1}{2}mn^2 + \frac{5}{2}mn - 2m - \frac{1}{6}n^3 - \frac{7}{4}n^2 + \frac{23}{12}n
\end{split}
$$

Vzhledem k tomu, že $m\geq n$, pak
$$
\begin{split}
počet_{celkem} &\leq \frac{1}{2}m^3 + \frac{5}{2}m^2 - 2m - \frac{1}{6}m^3 - \frac{7}{4}m^2 + \frac{23}{12}m \\
               &= \frac{1}{3}m^3 - \frac{3}{4}m^2 - \frac{1}{12}m \\
               &\in O(\frac{1}{3}m^3)
\end{split}
$$
Algoritmus má tedy asymtotickou časovou složitost $O(\frac{1}{3}m^3)$, tedy $\tilde{k} = \frac{1}{3}$ a $\tilde{l} = 3$.